This notebook presents the final calculation from the 
[growth_with_boundary_layer](./growth_with_boundary_layer.ipynb) notebook
but with the problem formally cast in terms of a pair of coupled ODEs solved
as an initial value problem using `scipy.integrate.solve_ivp`. The equations 
are briefly restated first.

We seek to find how the location of the particle ($l_p$) and it's radius ($r_p$) change with time.
The change in location is given by the Stokes equation: 

$$ \frac{d l_p}{d t } = \frac{2}{9} \frac{\Delta \rho g r_p^2}{\mu}$$

and the growth rate is determind from the liquid composition at the interface ($x_p$) and how fast oxygen
can be diffuse away from the interface:

$$\frac{\mathrm{d} r_p}{\mathrm{d} t} = \frac{D_l}{x_p} \left. \frac{\partial x}{\partial r}\right|_{r=r_p}  $$

The composition, $x$, is given by:

$$ x(r) = \begin{cases} 
      1.0 & r \leq r_p  \\
      (x_p - x_l)(r - r_p)/\delta + x_p & r_p\leq r\leq r_p + \delta \\
      x_l & r_p + \delta \leq r 
   \end{cases}$$
   
and we constrain the total composition around the particle (including the particle itself)
to match some imposed initial composition $x_i$:

$$ \frac{4}{3}\pi r^3 x_i = \int_0^R 4 \pi r^2 x(r) \mathrm{d}r$$

The thickness of the boundary layer depends on the falling velocity and radius of the particle in
a way that depends on $Re$:

$$ \delta = \begin{cases}
   2 r_p & Re \leq 10^{-2} \\
   2 r_p Pe^{-1/3} & 10^{-2} \leq Re \leq 10^2 \\
   2 r_p Re^{-1/2} Sc^{-1/3} & 200 \leq Re
   \end{cases} $$

with $Pe = (2r \frac{d l_p}{d t }) / D_l$, $Sc = \mu / D_l$ and $Re = (2 r_p \frac{d l_p}{d t }) / \mu$.

But then we have the whole thermodynamics stuff to put in two, which makes it hard to write down. Let's just 
try the code...



In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numba
import scipy.integrate as spi
import scipy.optimize as spo

import growth
import falling
import feo_thermodynamics

Pr =  0.09075  Sc =  999.9999999999999


In [2]:
@numba.vectorize()
def x_r_well_mixed (r, rp, xl):
    if r < rp:
        x = 1.0
    else:
        x = xl
    return x

@numba.jit()
def intergrand_well_mixed(r, rp, xl):
    return 4.0 * np.pi * r**2 * x_r_well_mixed(r, rp, xl)

def total_composition_well_mixed(rp, xl, r_max):
    y, err, infodict = spi.quad(intergrand_well_mixed, 0.0, r_max, args=(rp, xl), full_output=1)
    
    return y

def _total_composition_well_mixed_error(xl, rp, xi, rtot):
    """
    For a given xl compute the difference between the composition and the pure
    liquid composition. When this is zero we have a consistent solution
    """
    # Calculate total composition for this configuration
    xtot = total_composition_well_mixed(rp, xl, rtot)
    xtot_pure_melt = 4/3 * np.pi * rtot**3 * xi
    error = xtot - xtot_pure_melt
    return error

@numba.vectorize()
def x_r (r, rp, delta, xl, xp):
    if r < rp:
        comp = 1.0
    elif r < (rp + delta):
        comp = (xl - xp)*(r - rp)/delta + xp
    else:
        comp = xl
    return comp

@numba.jit()
def intergrand(r, rp, delta, xl, xp):
    return 4.0 * np.pi * r**2 * x_r(r, rp, delta, xl, xp)

def total_composition(rp, delta, xl, xp, r_max):
    # specifing function discontiunities as breakpoints reduces errors here.
    y, err = spi.quad(intergrand, 0.0, r_max, args=(rp, delta, xl, xp), points=(rp, rp+delta))
    return y, err

def _total_composition_error(xp, rp, xi, delta, rtot, temperature, pressure, dl, k0, debug=False):
    """
    For a given cl compute the difference between the composition and the pure
    liquid composition. When this is zero we have a consistent solution
    """
    # Compute growth rate for this composition 
    v = growth.growth_velocity_feo(xp, pressure, temperature, k0)
    # This gives us the composition at the edge of the boundary layer
    # because the graident at the boundary (and hence in the layer)
    # is set by the expulsion rate of O from the growing Fe
    xl = xp + (delta*xp)/dl * v # Check sign here - it's negative in the notes.
    # but oxygen content has changed sign.
    # Calculate total composition for this configuration
    xtot, integration_error = total_composition(rp, delta, xl, xp, rtot)
    xtot_pure_melt = 4/3 * np.pi * rtot**3 * xi
    error = xtot - xtot_pure_melt
    if debug:
        print("Composition error:", error, "intgration error:", integration_error)
    return error

def diffusion_growth_rate(rp, xi, delta, rtot, temperature, pressure, dl, k0, debug=False):
    """
    Compute growth rate of a particle of Fe from an FeO liquid accounting for a diffusional boundary layer
    
    For an Fe particle of radius rp (m) in a spherical container of radius rtot (m)
    calculate drp/dt (in m/s) assuming the presence of a linear boundary layer of
    thickness delta (m) and total composition ci (mol frac Fe) pressute (in GPa) and
    temperature (K). We also need two material properties, the diffusivity of FeO in 
    the liquid (dl, in m^2s^-1) and prefactor for growth (k0, in m/s). We also need
    an initial guess for the liquid composition next to the particle (cl_guess).
    
    Returns the growth rate, the self-consistent composition at the interface
    and the self consistent composition at the outer side of the boundary layer.
    """
    xp, root_result = spo.brentq(_total_composition_error, 1.0E-12, 1.0-1.0E-12, 
                                 args=(rp, xi, delta, rtot, temperature, pressure, dl, k0, debug),
                                 xtol=2.0e-14, disp=True, full_output=True)
    if debug:
        print(root_result)
    v = growth.growth_velocity_feo(xp, pressure, temperature, k0)
    xl = xp + (delta*xp)/dl * v
    error = _total_composition_error(xp, rp, xi, delta, rtot, temperature, pressure, dl, k0)
    return v, xp, xl, error

In [3]:
def dy_by_dt(t, y, xi, rtot, temperature, pressure, dl, k0, g, mu):
    """
    Find the growth rate and sinking rate of a particle at time t
    
    This is to be used with scipy.integrate.solve_ivp. t is the time
    (not used, everything is constant in time), y[0] is the current 
    particle radius and y[1] is the current height above the ICB.
    Returns growth rate of particle and vertical velocity of particle
    (positive is upwards). 
    """
    rp = y[0]
    z = y[1]
    print(t, 's, at z=', z, ',rp = ', rp)
    # Find liquid composition assuming no boundary layer (or we could do the whole thing
    # inside yet another self conssitent loop - no thanks).
    xl_no_bl = spo.brentq(_total_composition_well_mixed_error, 0.00000001, 0.999999999, 
                          args=(rp, xi, rtot))
    
    # Density calculation (with composition at previous step...)
    rho_liq, _, _, rho_hcp, _, _ = feo_thermodynamics.densities(xl_no_bl, pressure, temperature)
    delta_rho = rho_hcp - rho_liq
    
    # Falling velocity
    v_falling  = spo.brentq(falling.fzhang_opt, -1.0, 100.0, 
                             args=(rp, mu, g, delta_rho, rho_liq))
    # Boundary layers
    _, _, _, _, delta, _, _ = falling.calculate_boundary_layers([rp], mu, g, delta_rho, rho_liq, 
                                                    1.0, dl, radius_top_flayer, radius_inner_core)
    
    delta = delta[0] # Proper vectoriziation of calc boundary layers needed
    # Particle growth rate
    v, xp, xl, error = diffusion_growth_rate(rp, xi, delta, rtot, temperature, pressure, dl, k0)
    return [v, v_falling]

In [6]:
r0 = 1.0E-10 # initial radius, m
t = 5000.0 # temperature, K
p = 330.0 # GPa
xi = 0.95 # Overall composition, mol frac Fe
k0 = 150.0 # growth rate prefactor, m/s
rtot = 1.0 # initial box radius
dl = 1.0E-9 # diffusion
radius_inner_core = 1221.0e3
radius_top_flayer = radius_inner_core + 200.0e3
mu = 1.0e-6 # kinematic viscosity
g = 3.7 # ICB gravity - from PREM

sol = spi.solve_ivp(dy_by_dt, [0, 100000], [r0, radius_top_flayer], args=(xi, rtot, t, p, dl, k0, g, mu))

0.0 s, at z= 1421000.0 ,rp =  1e-10
9.652233010271119e-06 s, at z= 1421000.0 ,rp =  1.000010099296275e-05
0.00019304466020542239 s, at z= 1421000.0 ,rp =  0.000200000119859255
0.00028956699030813356 s, at z= 1421000.000000224 ,rp =  7.500054315668598e-05
0.0007721786408216895 s, at z= 1420999.9999968156 ,rp =  0.0009777777866923686
0.0008579762675796549 s, at z= 1420999.9999866525 ,rp =  0.002952595641615335
0.0009652233010271118 s, at z= 1420999.9999844653 ,rp =  0.0028462725526942167
0.0009652233010271118 s, at z= 1421000.000000752 ,rp =  9.114759863704727e-05
0.0011469478794952465 s, at z= 1421000.0000007935 ,rp =  9.114797080960489e-05
0.001237810168729314 s, at z= 1421000.0000008142 ,rp =  9.114815689588371e-05
0.0016921216148996507 s, at z= 1421000.0000009176 ,rp =  9.114908732727779e-05
0.0017728880942188218 s, at z= 1421000.000000936 ,rp =  9.11492527373034e-05
0.0018738461933677855 s, at z= 1421000.000000959 ,rp =  9.114945949983542e-05
0.0018738461933677855 s, at z= 1421000.0